In [1]:
# import required libriaries
from pathlib import Path
import json

from fpl_predictor.util import *

In [2]:
# path to data directory
#path = Path('/home/jupyter/fpl-prediction/data')
path = Path('./fpl_predictor/data')

In [3]:
# read in latest training set
df_train = pd.read_csv(path/'train_v8.csv', index_col=0, 
                       dtype={'season':str,
                              'comp':str,
                              'squad':str})

# path to current season
season_paths = [path/'2021-22']

# name for current season
season_names = ['2122']

# team codes
teams = pd.read_csv(path/'teams.csv')

# latest remaining season dataset, for latest week's play_proba values
remaining_season = pd.read_csv(path/'remaining_season.csv', index_col=0)

# create players dataset
all_players = build_players(path, season_paths, season_names, teams)

# 2019-20 market value
# table was in slightly different position for 2019-20 season
# should check that 2019-20 season values remain the same once season starts
teams_mv = build_season_mv('2122', 11, range(13,33))

In [4]:
# all_players#[all_players['full_name'] == 'Ben Davies']

In [5]:
# df_train = df_train.iloc[:-686]

In [6]:
# find the latest gameweek
last_gw = df_train['gw'][df_train['season'] == season_names[0]].max()

if np.isnan(last_gw): 
    last_gw = 1 
else: 
    last_gw = last_gw + 1
    
last_gw

32

In [7]:
# build training set for latest gameweek
df_latest = build_season(season_paths[0], season_names[0], all_players, teams, teams_mv, [last_gw])

In [8]:
df_latest.drop_duplicates(inplace=True)

In [9]:
# df_latest[df_latest['player'] == 'Chris Wood']
# lucas digne
# chris wood

In [10]:
last_play_proba = remaining_season[remaining_season['gw'] == last_gw][['player', 'team', 'play_proba']]

In [11]:
last_play_proba.head()

,player,team,play_proba
0,Joseph Willock,Newcastle United,0.75
1,Matt Targett,Newcastle United,1.00
2,Dan Burn,Newcastle United,1.00
3,Chris Wood,Newcastle United,1.00
4,Jonjo Shelvey,Newcastle United,1.00


In [12]:
# add latest week's play_proba for each player
df_latest = df_latest.merge(last_play_proba, on=['player', 'team'], how='left')

In [13]:
df_latest.drop_duplicates(inplace=True)

In [14]:
df_train.drop_duplicates(inplace=True)

In [15]:
if last_gw==1:
    df_latest['relative_market_value_team_season'] = df_latest['relative_market_value_team']
    df_latest['relative_market_value_opponent_team_season'] = df_latest['relative_market_value_opponent_team']

In [16]:
# for when you're doing the season value properly
# relative_market_value_season = train_df[train_df['season'] == '2021'][['team', 'relative_market_value_team']].groupby(['team']).first()
# relative_market_value_season = relative_market_value_season.rename(columns={'relative_market_value_team': 'season_value'})
# relative_market_value_season

In [17]:
# now get futher stats from fbref
# standard stats
features = ['squad', 'comp', 'round', 'shots_total', 'shots_on_target', 'touches', 'pressures',
            'tackles', 'interceptions', 'blocks', 'xg', 'npxg', 'xa', 'sca', 
            'gca', 'passes_completed', 'passes', 'passes_pct',  'carries', 
            'dribbles_completed', 'dribbles']

In [18]:
# url for the premier league
url_base = 'https://fbref.com'
pl_history_page = '/en/comps/9/history/Premier-League-Seasons'
url = url_base + pl_history_page

# scrape the history table to get latest season
pl_history_table = get_table(url)
pl_history_urls = get_urls(['season'], 'season', pl_history_table, url_base)
season_urls = pl_history_urls[0:1]

df_enhanced = get_stats(features, season_urls, url_base)

2021-2022
https://fbref.com/en/comps/9/Premier-League-Stats
Manchester City
https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats
Ederson
João Cancelo
Bernardo Silva
Rodri
Aymeric Laporte
Rúben Dias
Phil Foden
Kyle Walker
Raheem Sterling
Kevin De Bruyne
Jack Grealish
Gabriel Jesus
İlkay Gündoğan
Riyad Mahrez
John Stones
Nathan Aké
Fernandinho
Oleksandr Zinchenko
Ferrán Torres
Cole Palmer
Zack Steffen
Benjamin Mendy
James Mcatee
Liam Delap
Kayky Chagas
Liverpool
https://fbref.com/en/squads/822bd0ba/Liverpool-Stats
Alisson
Virgil van Dijk
Mohamed Salah
Trent Alexander-Arnold
Sadio Mané
Joël Matip
Jordan Henderson
Andrew Robertson
Fabinho
Diogo Jota
Thiago Alcántara
Naby Keïta
Roberto Firmino
Alex Oxlade-Chamberlain
Ibrahima Konaté
Curtis Jones
Kostas Tsimikas
James Milner
Luis Díaz
Harvey Elliott
Joe Gomez
Caoimhín Kelleher
Tyler Morton
Takumi Minamino
Divock Origi
Kaide Gordon
Neco Williams
Chelsea
https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats
Antonio Rüdiger
Edouard Mendy
Thi

Juraj Kucka
Ismaila Sarr
Tom Cleverley
William Troost-Ekong
Imran Louza
Samir Santos
Hassane Kamara
Cucho
Daniel Bachmann
Adam Masina
João Pedro
Danny Rose
Christian Kabasele
Jeremy Ngakia
Edo Kayembe
Francisco Sierralta
Ken Sema
Oghenekaro Etebo
Ozan Tufan
Nicolas Nkoulou
Samuel Kalu
Ashley Fletcher
Troy Deeney
Dan Gosling
Norwich City
https://fbref.com/en/squads/1c781004/Norwich-City-Stats
Teemu Pukki
Kenny McLean
Max Aarons
Ben Gibson
Grant Hanley
Tim Krul
Brandon Williams
Pierre Lees-Melou
Milot Rashica
Josh Sargent
Billy Gilmour
Mathias Normann
Ozan Kabak
Dimitris Giannoulis
Angus Gunn
Adam Idah
Lukas Rupp
Przemysław Płacheta
Kieran Dowell
Todd Cantwell
Sam Byram
Andrew Omobamidele
Jakob Sørensen
Christos Tzolis
Christoph Zimmermann
Jon Rowe
Bali Mumba


In [19]:
# get the player name matching dictionary
with open(path/'match_list.json') as f:
  match_list = json.load(f)

match_list

{'Fabinho': 'Fabio Henrique Tavares',
 'Alisson': 'Alisson Ramses Becker',
 'Joël Matip': 'Joel Matip',
 'Joe Gomez': 'Joseph Gomez',
 'Naby Keïta': 'Naby Keita',
 'Adrián': 'Adrián San Miguel del Castillo',
 'Thiago Alcántara': 'Thiago Alcántara do Nascimento',
 'Allan': 'Allan Marques Loureiro',
 'Richarlison': 'Richarlison de Andrade',
 'Gylfi Sigurðsson': 'Gylfi Sigurdsson',
 'Séamus Coleman': 'Seamus Coleman',
 'André Gomes': 'André Filipe Tavares Gomes',
 'Bernard': 'Bernard Anício Caldeira Duarte',
 'Çağlar Söyüncü': 'Çaglar Söyüncü',
 'Bruno Fernandes': 'Bruno Miguel Borges Fernandes',
 'Fred': 'Frederico Rodrigues de Paula Santos',
 'Nemanja Matić': 'Nemanja Matic',
 'Alex Telles': 'Alex Nicolao Telles',
 'Pierre Højbjerg': 'Pierre-Emile Højbjerg',
 'Son Heung-min': 'Heung-Min Son',
 'Lucas Moura': 'Lucas Rodrigues Moura da Silva',
 'Érik Lamela': 'Erik Lamela',
 'Dele Alli': 'Bamidele Alli',
 'Carlos Vinícius': 'Carlos Vinicius Alves Morais',
 'Ederson': 'Ederson Santana de M

In [20]:
team_list = {}
team_list['Brighton'] = 'Brighton and Hove Albion'
team_list['Leeds United'] = 'Leeds'
team_list['Manchester Utd'] = 'Manchester United'
team_list['Newcastle Utd'] = 'Newcastle United'
team_list['Norwich City'] = 'Norwich'
team_list['Sheffield Utd'] = 'Sheffield United'
team_list['Tottenham'] = 'Tottenham Hotspur'
team_list['West Brom'] = 'West Bromwich Albion'
team_list['West Ham'] = 'West Ham United'
team_list['Wolves'] = 'Wolverhampton Wanderers'

In [21]:
df_enhanced = df_enhanced.loc[df_enhanced[df_enhanced['round'].str[-2:] != 'on'].index]
df_enhanced = df_enhanced.loc[df_enhanced[df_enhanced['squad'] != 'CSKA Moscow'].index]

In [22]:
# replace non-matching names/teams and extract gw as integer
df_enhanced['alt_name'] = df_enhanced['player'].replace(match_list)
df_enhanced['squad'] = df_enhanced['squad'].replace(team_list)
df_enhanced['round'] = df_enhanced['round'].str[-2:].astype('int')

In [23]:
# remove duplicated (happens when players transferred within season)
df_enhanced = df_enhanced.drop_duplicates()

In [24]:
df_enhanced

,season,player,date,squad,comp,round,shots_total,shots_on_target,touches,pressures,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,alt_name
0,2021-2022,Ederson,2021-08-15,Manchester City,Premier League,1,0,0,22,0,...,0.0,2,0,20,22,90.9,10,0,0,Ederson Santana de Moraes
1,2021-2022,Ederson,2021-08-21,Manchester City,Premier League,2,0,0,14,0,...,0.0,0,0,12,12,100.0,6,0,0,Ederson Santana de Moraes
2,2021-2022,Ederson,2021-08-28,Manchester City,Premier League,3,0,0,37,0,...,0.0,0,0,31,37,83.8,25,0,0,Ederson Santana de Moraes
3,2021-2022,Ederson,2021-09-11,Manchester City,Premier League,4,0,0,26,0,...,0.0,0,0,22,24,91.7,10,0,0,Ederson Santana de Moraes
4,2021-2022,Ederson,2021-09-18,Manchester City,Premier League,5,0,0,40,0,...,0.0,0,0,39,39,100.0,28,0,0,Ederson Santana de Moraes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8726,2021-2022,Jon Rowe,2022-03-05,Norwich,Premier League,28,0,0,17,6,...,0.0,2,1,9,13,69.2,13,0,0,Jonathan Rowe
8727,2021-2022,Jon Rowe,2022-03-10,Norwich,Premier League,30,0,0,8,0,...,0.0,0,0,7,7,100.0,8,0,0,Jonathan Rowe
8728,2021-2022,Jon Rowe,2022-03-13,Norwich,Premier League,29,2,1,15,6,...,0.0,0,0,6,9,66.7,13,2,3,Jonathan Rowe
8729,2021-2022,Jon Rowe,2022-04-02,Norwich,Premier League,31,0,0,8,8,...,0.0,0,0,1,4,25.0,6,0,1,Jonathan Rowe


In [25]:
# create date field in same format as enhanced data
df_latest['date'] = df_latest['kickoff_time'].str[:10]

In [26]:
# check if any names aren't matching
enhanced_players = df_enhanced[df_enhanced['date'].isin(df_latest['date'].unique())]['alt_name'].unique().tolist()
standard_players = df_latest['player'].unique().tolist()

unmatched_players = [x for x in enhanced_players if x not in standard_players]
unmatched_players

[]

In [27]:
# check if any teams aren't matching
enhanced_teams = df_enhanced[df_enhanced['date'].isin(df_latest['date'].unique())]['squad'].unique().tolist()
standard_teams = df_latest['team'].unique().tolist()

unmatched_teams = [x for x in enhanced_teams if x not in standard_teams]
unmatched_teams

[]

In [28]:
# any additional corrections
# match_list['Tariqe Fosu'] = 'Tariqe Fosu-Henry'
# match_list['Josh Dasilva'] = 'Pelenda Joshua Dasilva'
# match_list['Bruno Guimarães'] = 'Bruno Guimarães Rodriguez Moura'
# match_list['Samir Santos'] = 'Samir Caetano de Souza Santos'
# match_list['Hwang Hee-chan'] = 'Hee-Chan Hwang'

# match_list['Sadio Man�'] = 'Sadio Mané'
# match_list['Jo�l Matip'] = 'Joel Matip'
# match_list['Thiago Alc�ntara'] = 'Thiago Alcántara do Nascimento'
# match_list['Naby Ke�ta'] = 'Naby Keita'
# match_list['Luis D�az'] = 'Luis Díaz'
# match_list['Jos� S�'] = 'José Malheiro de Sá'
# match_list['Romain Sa�ss'] = 'Romain Saïss'
# match_list['N�lson Semedo'] = 'Nélson Cabral Semedo'
# match_list['Ra�l Jim�nez'] = 'Raúl Jiménez'
# match_list['Jo�o Moutinho'] = 'João Filipe Iria Santos Moutinho'
# match_list['R�ben Neves'] = 'Rúben Diogo da Silva Neves'
# match_list['Rayan A�t Nouri'] = 'Rayan Ait Nouri'
# match_list['Fernando Mar�al'] = 'Fernando Marçal'
# match_list['Emiliano Mart�nez'] = 'Emiliano Martínez'
# match_list['Emi Buend�a'] = 'Emiliano Buendía Stati'
# match_list['Fabian Sch�r'] = 'Fabian Schär'
# match_list['Martin D�bravka'] = 'Martin Dubravka'
# match_list['Miguel Almir�n'] = 'Miguel Almirón'
# match_list['Bruno Guimar�es'] = 'Bruno Guimarães Rodriguez Moura'
# match_list['Kiko Femen�a'] = 'Francisco Femenía Far'

In [29]:
# replace additional non-matching names
# df_enhanced['alt_name'] = df_enhanced['player'].replace(match_list)

In [30]:
# make sure there are no non matches after update
# enhanced_teams = df_enhanced[df_enhanced['date'].isin(df_latest['date'].unique())]['squad'].unique().tolist()
# standard_teams = df_latest['team'].unique().tolist()

# unmatched_teams = [x for x in enhanced_teams if x not in standard_teams]
# unmatched_teams

In [31]:
# if match list was updated then update the file
# json_file = json.dumps(match_list)
# f = open(path/"match_list.json","w")
# f.write(json_file)
# f.close()

In [32]:
# check there are no duplicates in the join fields for the enhanced data
assert(len(df_enhanced[df_enhanced.duplicated(subset=['alt_name', 'date'], keep=False)]) == 0)

In [33]:
df_latest[df_latest.duplicated(subset=['date','player','team'], keep=False)]

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba,date


In [34]:
# check there are no duplicates in the join fields for the standard data
assert(len(df_train[df_train.duplicated(subset=['date','player','team'])]) == 0)

In [35]:
# df_train[df_train.duplicated(subset=['date','player','team'])]

In [36]:
# join standard and enhanced dataframes
df_latest_further = df_latest.merge(df_enhanced.drop(['player', 'season', 'round'], axis=1), 
                                   left_on=['date', 'player', 'team'], 
                                   right_on=['date', 'alt_name', 'squad'], how='left')

In [37]:
# drop the alt name field and check the length of the standard
# dataframe has not increased
df_latest_further = df_latest_further.drop(['alt_name'], axis=1)
assert(len(df_latest_further) == len(df_latest_further))

In [38]:
# add crowds field
df_latest_further['crowds'] = 1

In [39]:
assert(sum(df_latest_further[df_latest_further['minutes'] > 0]['xg'].isnull()) == 0)

In [40]:
df_latest_further[df_latest_further['minutes'] > 0][df_latest_further[df_latest_further['minutes'] > 0]['xg'].isnull()]

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds


In [41]:
df_enhanced

,season,player,date,squad,comp,round,shots_total,shots_on_target,touches,pressures,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,alt_name
0,2021-2022,Ederson,2021-08-15,Manchester City,Premier League,1,0,0,22,0,...,0.0,2,0,20,22,90.9,10,0,0,Ederson Santana de Moraes
1,2021-2022,Ederson,2021-08-21,Manchester City,Premier League,2,0,0,14,0,...,0.0,0,0,12,12,100.0,6,0,0,Ederson Santana de Moraes
2,2021-2022,Ederson,2021-08-28,Manchester City,Premier League,3,0,0,37,0,...,0.0,0,0,31,37,83.8,25,0,0,Ederson Santana de Moraes
3,2021-2022,Ederson,2021-09-11,Manchester City,Premier League,4,0,0,26,0,...,0.0,0,0,22,24,91.7,10,0,0,Ederson Santana de Moraes
4,2021-2022,Ederson,2021-09-18,Manchester City,Premier League,5,0,0,40,0,...,0.0,0,0,39,39,100.0,28,0,0,Ederson Santana de Moraes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8726,2021-2022,Jon Rowe,2022-03-05,Norwich,Premier League,28,0,0,17,6,...,0.0,2,1,9,13,69.2,13,0,0,Jonathan Rowe
8727,2021-2022,Jon Rowe,2022-03-10,Norwich,Premier League,30,0,0,8,0,...,0.0,0,0,7,7,100.0,8,0,0,Jonathan Rowe
8728,2021-2022,Jon Rowe,2022-03-13,Norwich,Premier League,29,2,1,15,6,...,0.0,0,0,6,9,66.7,13,2,3,Jonathan Rowe
8729,2021-2022,Jon Rowe,2022-04-02,Norwich,Premier League,31,0,0,8,8,...,0.0,0,0,1,4,25.0,6,0,1,Jonathan Rowe


In [42]:
df_train_new = pd.concat([df_train, df_latest_further], ignore_index=True, axis=0, sort=False)

In [43]:
df_train_new[df_train_new['player'] == 'Harry Kane'].tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds
132634,Harry Kane,29,4,90,Tottenham Hotspur,Manchester United,NaN,NaN,False,6,...,0.0,3.0,0.0,17.0,28.0,60.7,27.0,1.0,3.0,1
132635,Harry Kane,29,4,90,Tottenham Hotspur,Brighton and Hove Albion,NaN,NaN,False,8,...,0.3,7.0,0.0,23.0,32.0,71.9,40.0,2.0,4.0,1
133347,Harry Kane,30,4,90,Tottenham Hotspur,West Ham United,NaN,NaN,True,13,...,0.7,6.0,2.0,26.0,42.0,61.9,47.0,3.0,3.0,1
133813,Harry Kane,31,4,90,Tottenham Hotspur,Newcastle United,NaN,NaN,True,4,...,0.9,7.0,2.0,16.0,25.0,64.0,27.0,1.0,2.0,1
134575,Harry Kane,32,4,90,Tottenham Hotspur,Aston Villa,NaN,NaN,False,10,...,0.6,5,3,15,18,83.3,24,3,6,1


In [44]:
df_train_new.tail(10)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds
135043,Yerry Mina,32,2,0,Everton,Manchester United,NaN,NaN,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
135044,Yerson Mosquera Valdelamar,32,2,0,Wolverhampton Wanderers,Newcastle United,NaN,NaN,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
135045,Yoane Wissa,32,3,90,Brentford,West Ham United,NaN,NaN,True,3,...,0.1,2,0,9,13,69.2,17,0,4,1
135046,Youri Tielemans,32,3,90,Leicester City,Crystal Palace,NaN,NaN,True,2,...,0.0,1,1,47,52,90.4,41,1,1,1
135047,Yves Bissouma,32,3,73,Brighton and Hove Albion,Arsenal,NaN,NaN,False,2,...,0.0,0,0,17,21,81.0,16,0,0,1
135048,Zach Awe,32,2,0,Arsenal,Brighton and Hove Albion,NaN,NaN,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
135049,Zack Steffen,32,1,0,Manchester City,Liverpool,NaN,NaN,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
135050,Zak Swanson,32,2,0,Arsenal,Brighton and Hove Albion,NaN,NaN,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
135051,Álvaro Fernández,32,1,0,Brentford,West Ham United,NaN,NaN,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
135052,Çaglar Söyüncü,32,2,90,Leicester City,Crystal Palace,NaN,NaN,True,2,...,0.0,2,0,70,78,89.7,64,0,1,1


In [45]:
df_latest_further.shape

(722, 57)

In [46]:
df_train_new.shape

(135053, 59)

In [47]:
# save latest training set to csv
# overwrite existing one
df_train_new.to_csv(path/'train_v8.csv')

In [48]:
# get fixtures
fixtures = pd.read_csv(path/'2021-22/fixtures.csv')
team_dict = teams[['team_2122', 'team']].dropna().set_index('team_2122').to_dict()['team']

fixtures['home_team'] = fixtures['team_h'].replace(team_dict)
fixtures['away_team'] = fixtures['team_a'].replace(team_dict)
fixtures = fixtures[['home_team', 'away_team', 'kickoff_time', 'event']]
fixtures = fixtures.rename(columns={'event':'gw'})


# use these two lines if updating play_proba later in week
# current_gw = last_gw
# df_train_new = pd.read_csv(path/'train_v7.csv', index_col=0, dtype={'season':str,
#                                                                     'comp':str,
#                                                                     'squad':str})

# set starting gameweek (where are we right now in the season)
current_gw = last_gw + 1
fixtures = fixtures[fixtures['gw'] >= current_gw].sort_values(by=['gw'])
fixtures['match_no'] = np.arange(fixtures.shape[0])

In [49]:
fixtures.head(50)

,home_team,away_team,kickoff_time,gw,match_no
310,Tottenham Hotspur,Brighton and Hove Albion,2022-04-16T11:30:00Z,33.0,0
311,Manchester United,Norwich,2022-04-16T14:00:00Z,33.0,1
312,Southampton,Arsenal,2022-04-16T14:00:00Z,33.0,2
313,Watford,Brentford,2022-04-16T14:00:00Z,33.0,3
314,Newcastle United,Leicester City,2022-04-17T13:15:00Z,33.0,4
315,West Ham United,Burnley,2022-04-17T13:15:00Z,33.0,5
316,Liverpool,Manchester United,2022-04-19T19:00:00Z,33.0,6
317,Chelsea,Arsenal,2022-04-20T18:45:00Z,33.0,7
318,Everton,Leicester City,2022-04-20T18:45:00Z,33.0,8
319,Newcastle United,Crystal Palace,2022-04-20T18:45:00Z,33.0,9


In [50]:
# add universal team codes for home and away teams
fixtures = fixtures.merge(teams, left_on='home_team', right_on='team', how='left')
fixtures = fixtures.merge(teams, left_on='away_team', right_on='team', how='left')
fixtures = fixtures[['gw', 'match_no', 'home_team', 'away_team', 'kickoff_time', 'team_code_x', 'team_code_y']]
fixtures.rename(index=str,
                columns={'team_code_x':'home_team_code',
                         'team_code_y':'away_team_code'},
                inplace=True)

In [51]:
fixtures.head()

,gw,match_no,home_team,away_team,kickoff_time,home_team_code,away_team_code
0,33.0,0,Tottenham Hotspur,Brighton and Hove Albion,2022-04-16T11:30:00Z,6,36
1,33.0,1,Manchester United,Norwich,2022-04-16T14:00:00Z,1,45
2,33.0,2,Southampton,Arsenal,2022-04-16T14:00:00Z,20,3
3,33.0,3,Watford,Brentford,2022-04-16T14:00:00Z,57,94
4,33.0,4,Newcastle United,Leicester City,2022-04-17T13:15:00Z,4,13


In [52]:
# all_players.replace(to_replace='None', value=1, inplace=True)

In [53]:
all_players[all_players['play_proba_2122'] == '0']

,full_name,play_proba_2122,position_2122,id_2122,cost_2122,team_2122,web_name_2122
1,Rúnar Alex Rúnarsson,0,1,2,40,3,Rúnarsson
2,Willian Borges Da Silva,0,3,3,63,3,Willian
3,Pierre-Emerick Aubameyang,0,4,4,96,3,Aubameyang
7,Pablo Marí,0,2,8,42,3,Marí
8,Héctor Bellerín,0,2,9,48,3,Bellerín
...,...,...,...,...,...,...,...
712,Christian Marques,0,2,549,40,39,Marques
714,Louie Moulden,0,1,611,40,39,Moulden
717,Bruno André Cavaco Jordão,0,3,661,45,39,Jordão
719,Dion Sanderson,0,2,686,40,39,Sanderson


In [54]:
# join home team to all players for current season
home_df = fixtures.merge(all_players, 
               left_on='home_team_code', 
               right_on='team_2122', 
               how='left')

# pull out the required fields and rename columns
home_df = home_df[['gw', 'match_no', 'home_team', 'away_team', 'kickoff_time', 
                   'full_name', 'position_2122', 'cost_2122', 'play_proba_2122', 
                   'web_name_2122']]
home_df.rename(index=str, 
               columns={'home_team':'team',
                        'away_team':'opponent_team',
                        'full_name':'player',
                        'position_2122':'position',
                        'cost_2122':'price',
                        'play_proba_2122':'play_proba',
                        'web_name_2122':'short_name'},
              inplace=True)

# add home flag
home_df['was_home'] = True

In [55]:
# join away team to all players for current season
away_df = fixtures.merge(all_players, 
               left_on='away_team_code', 
               right_on='team_2122', 
               how='left')

# pull out the required fields and rename columns
away_df = away_df[['gw',  'match_no', 'away_team', 'home_team', 'kickoff_time', 
                   'full_name', 'position_2122', 'cost_2122', 'play_proba_2122', 
                   'web_name_2122']]
away_df.rename(index=str, 
               columns={'away_team':'team',
                        'home_team':'opponent_team',
                        'full_name':'player',
                        'position_2122':'position',
                        'cost_2122':'price',
                        'play_proba_2122':'play_proba',
                        'web_name_2122':'short_name'},
              inplace=True)

# add home flag
away_df['was_home'] = False

In [56]:
# look at away players
away_df.head()

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home
0,33.0,0,Brighton and Hove Albion,Tottenham Hotspur,2022-04-16T11:30:00Z,Adam Lallana,3,53,100,Lallana,False
1,33.0,0,Brighton and Hove Albion,Tottenham Hotspur,2022-04-16T11:30:00Z,Jason Steele,1,39,100,Steele,False
2,33.0,0,Brighton and Hove Albion,Tottenham Hotspur,2022-04-16T11:30:00Z,Pascal Groß,3,57,100,Groß,False
3,33.0,0,Brighton and Hove Albion,Tottenham Hotspur,2022-04-16T11:30:00Z,Shane Duffy,2,40,50,Duffy,False
4,33.0,0,Brighton and Hove Albion,Tottenham Hotspur,2022-04-16T11:30:00Z,Davy Pröpper,3,45,0,Pröpper,False


In [57]:
# look at home players
home_df.head()

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home
0,33.0,0,Tottenham Hotspur,Brighton and Hove Albion,2022-04-16T11:30:00Z,Ben Davies,2,44,100,Davies,True
1,33.0,0,Tottenham Hotspur,Brighton and Hove Albion,2022-04-16T11:30:00Z,Joe Hart,1,45,0,Hart,True
2,33.0,0,Tottenham Hotspur,Brighton and Hove Albion,2022-04-16T11:30:00Z,Hugo Lloris,1,54,None,Lloris,True
3,33.0,0,Tottenham Hotspur,Brighton and Hove Albion,2022-04-16T11:30:00Z,Toby Alderweireld,2,50,0,Alderweireld,True
4,33.0,0,Tottenham Hotspur,Brighton and Hove Albion,2022-04-16T11:30:00Z,Erik Lamela,3,60,0,Lamela,True


In [58]:
# # calculate average minutes over last 5 gameweeks
# # first get last 5 gameweeks in current season
# recent_mins = df_train_new[(df_train_new['gw'] >= current_gw - 5) & 
#                            (df_train_new['season'] == season_names[0])][['player', 'minutes', 'play_proba']]

# # recent_mins = df_train_new[((df_train_new['gw'] >= 40 - 5) & 
# #                            (df_train_new['season'] == '1920')) | 
# #                            ((df_train_new['gw'] >= 1) & 
# #                            (df_train_new['season'] == '2021'))][['player', 'minutes', 'play_proba']]

# # exclude weeks where the player had an injury designation (i.e. keep where play_proba is 1)
# recent_mins = recent_mins[recent_mins['play_proba'] == 1]

# len(recent_mins)

# # average for each player
# recent_mins = recent_mins.groupby(['player'])['minutes'].mean().to_dict()

# recent_mins

In [59]:
# concatenate home and away players
remaining_season_df = home_df.append(away_df).reset_index(drop=True)

# add home and away team market values
remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '2122'],
                                                left_on='team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '2122'],
                                                left_on='opponent_team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df.rename(index=str, 
                           columns={'relative_market_value_x':'relative_market_value_team',
                                    'relative_market_value_y':'relative_market_value_opponent_team'},
                           inplace=True)

# add season name
remaining_season_df['season'] = season_names[0]

# divide cost by 10 for actual cost
remaining_season_df['price'] = remaining_season_df['price']/10

# set availability probability
# 0 = 0% chance, 25 = 25% chance, etc
# 'None' or '100' = 100% chance
remaining_season_df.loc[remaining_season_df['play_proba'] == 'None', 'play_proba'] = 100
remaining_season_df['play_proba'] = remaining_season_df['play_proba'].astype('float') / 100

# cast position to integer
remaining_season_df['position'] = remaining_season_df['position'].astype(int)

# # add minutes based on last five fixtures
# remaining_season_df['minutes'] = remaining_season_df['player'].map(recent_mins)
# remaining_season_df['minutes'].fillna(0, inplace=True)

# # multiply minutes by play probability
# remaining_season_df['minutes'] = remaining_season_df['minutes'] * remaining_season_df['play_proba']

In [60]:
remaining_season_df.head(50)

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home,relative_market_value_team,relative_market_value_opponent_team,season
0,33.0,0,Tottenham Hotspur,Brighton and Hove Albion,2022-04-16T11:30:00Z,Ben Davies,2,4.4,1.0,Davies,True,NaN,NaN,2122
1,33.0,0,Tottenham Hotspur,Brighton and Hove Albion,2022-04-16T11:30:00Z,Joe Hart,1,4.5,0.0,Hart,True,NaN,NaN,2122
2,33.0,0,Tottenham Hotspur,Brighton and Hove Albion,2022-04-16T11:30:00Z,Hugo Lloris,1,5.4,1.0,Lloris,True,NaN,NaN,2122
3,33.0,0,Tottenham Hotspur,Brighton and Hove Albion,2022-04-16T11:30:00Z,Toby Alderweireld,2,5.0,0.0,Alderweireld,True,NaN,NaN,2122
4,33.0,0,Tottenham Hotspur,Brighton and Hove Albion,2022-04-16T11:30:00Z,Erik Lamela,3,6.0,0.0,Lamela,True,NaN,NaN,2122
5,33.0,0,Tottenham Hotspur,Brighton and Hove Albion,2022-04-16T11:30:00Z,Harry Kane,4,12.6,1.0,Kane,True,NaN,NaN,2122
6,33.0,0,Tottenham Hotspur,Brighton and Hove Albion,2022-04-16T11:30:00Z,Serge Aurier,2,4.9,0.0,Aurier,True,NaN,NaN,2122
7,33.0,0,Tottenham Hotspur,Brighton and Hove Albion,2022-04-16T11:30:00Z,Heung-Min Son,3,11.1,1.0,Son,True,NaN,NaN,2122
8,33.0,0,Tottenham Hotspur,Brighton and Hove Albion,2022-04-16T11:30:00Z,Matt Doherty,2,4.9,0.0,Doherty,True,NaN,NaN,2122
9,33.0,0,Tottenham Hotspur,Brighton and Hove Albion,2022-04-16T11:30:00Z,Eric Dier,2,4.6,1.0,Dier,True,NaN,NaN,2122


In [61]:
# save latest prediction set to csv
remaining_season_df.to_csv(path/'remaining_season.csv')